Reading dataset

# Installs

In [4]:
! pip install pandas
! pip install matplotlib
! pip install pandas
! pip install numpy==1.23.5
! pip install scikit-surprise
! pip install scikit-learn
! pip install pyarrow

   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.1 MB 5.6 MB/s eta 0:00:05
   ---- ----------------------------------- 2.6/25.1 MB 7.9 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.1 MB 8.7 MB/s eta 0:00:03
   ---------- ----------------------------- 6.3/25.1 MB 8.8 MB/s eta 0:00:03
   ------------ --------------------------- 8.1/25.1 MB 8.8 MB/s eta 0:00:02
   ---------------- ----------------------- 10.2/25.1 MB 9.0 MB/s eta 0:00:02
   ------------------- -------------------- 12.3/25.1 MB 9.2 MB/s eta 0:00:02
   ---------------------- ----------------- 14.4/25.1 MB 9.3 MB/s eta 0:00:02
   -------------------------- ------------- 16.5/25.1 MB 9.5 MB/s eta 0:00:01
   ------------------------------ --------- 18.9/25.1 MB 9.7 MB/s eta 0:00:01
   -------------------------------- ------- 20.7/25.1 MB 9.6 MB/s eta 0:00:01
   ----------------------------------- ---- 22.5/25.1 MB 9.5 MB/s eta 0:00:01

# Imports

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from datetime import datetime
import numpy as np

# Preprocessing

Forming dataset

In [47]:
smm_train_data = pd.read_parquet('../Recomendation systems baseline/datasets/train_smm.parquet')
smm_test_data = pd.read_parquet('../Recomendation systems baseline/datasets/test_smm.parquet')
zvuk_train_data = pd.read_parquet('../Recomendation systems baseline/datasets/train_zvuk.parquet')
zvuk_test_data = pd.read_parquet('../Recomendation systems baseline/datasets/test_zvuk.parquet')


In [41]:
zvuk_train_data

,user_id,timestamp,item_id,rating
0,6486258,1673740800074000000,75474,1
1,7436216,1673740801368000000,24228,5
2,1736489,1673740801516000000,13113,3
3,4976699,1673740803052000000,76029,5
4,4976699,1673740803056000000,105990,1
...,...,...,...,...
30957055,3961121,1682478698807000000,95865,24
30957056,9067175,1682478698944000000,3615,1
30957057,4180391,1682478699230000000,1519,1
30957058,2088083,1682478699233000000,71478,1


In [52]:
def dataframe_preprocess(data):
    data['Time'] = pd.to_datetime(data['timestamp'])
    user_item_matrix = data.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)
    return data

Инфо о датафреймах

In [53]:
dataframe_preprocess(smm_train_data)

c:\Users\Ivan\PycharmProjects\sberrecsys\.venv\lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


IndexError: index 597965866 is out of bounds for axis 0 with size 597965504

In [25]:
zvuk_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24458793 entries, 0 to 24458792
Data columns (total 4 columns):
 #   Column     Dtype
---  ------     -----
 0   user_id    int64
 1   timestamp  int64
 2   item_id    int64
 3   rating     int64
dtypes: int64(4)
memory usage: 746.4 MB


In [24]:
smm_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18601324 entries, 0 to 18601323
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    int64  
 1   item_id    int64  
 2   timestamp  int64  
 3   rating     float64
dtypes: float64(1), int64(3)
memory usage: 567.7 MB


кол-во уникальных айдишников

In [20]:
print(zvuk_test_data.user_id.nunique())
print(zvuk_test_data.item_id.nunique())

118069
106634


время из unix в датетиме

In [33]:
zvuk_test_data['timestamp'] = pd.to_datetime(zvuk_test_data['timestamp'])
zvuk_test_data

,user_id,timestamp,item_id,rating
0,1736489,2023-01-15 00:00:01.516,13113,3
1,4976699,2023-01-15 00:00:03.052,76029,5
2,4976699,2023-01-15 00:00:03.056,105990,1
3,2393293,2023-01-15 00:00:03.556,22627,3
4,5477350,2023-01-15 00:00:03.965,49458,2
...,...,...,...,...
24458788,3961121,2023-04-26 03:11:38.807,95865,24
24458789,9067175,2023-04-26 03:11:38.944,3615,1
24458790,4180391,2023-04-26 03:11:39.230,1519,1
24458791,2088083,2023-04-26 03:11:39.233,71478,1


Prepro

In [14]:
data['Time'] = pd.to_datetime(data['Time'], unit='s')
user_item_matrix = data.pivot_table(index='UserId', columns='ProductId', values='Score').fillna(0)
user_item_sparse_matrix = csr_matrix(user_item_matrix.values)



c:\Users\Ivan\PycharmProjects\Recomendation systems baseline\.venv\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


IndexError: index 1834577734 is out of bounds for axis 0 with size 1834560038

# Pipeline and baseline

In [10]:
svd = TruncatedSVD(n_components=50, random_state=42)
user_matrix = svd.fit_transform(user_item_sparse_matrix)
item_matrix = svd.components_

NameError: name 'user_item_sparse_matrix' is not defined